In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from helpers import *
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from datetime import datetime
import seaborn as sns
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:

save_dir = ''
file_names = glob.glob(os.path.join(save_dir, f'QCEH_data/TCV_DATAno*.parquet'))

In [3]:
precomputed_dir = 'precomputed_data/'

List of dataframes (one discharge per each element of the list)

In [4]:
df_list = [pd.read_parquet(x) for x in file_names]
len(df_list)

60

# Initial Analysis - necessary values
For this project we are only focusing on the machine inputs provided in the data.

In [5]:
GENERAL_INPUTS = ["shotnumber", "time"]
MACHINE_INPUTS = ["isbaffled", "ip","b0","nel","ptot","pdiv","q95","betan","kappa","deltaavg","deltaupp",
                  "deltalow","gapin","gapout","zmag","rmag","rmin","lpar_ot","zeff"]
LABEL = ["LHD_label"]
INPUTS = GENERAL_INPUTS + MACHINE_INPUTS + LABEL
len(INPUTS)

22

In [71]:
# This normalizes the X values.
X_train, y_train, column_names = get_training_data(include_time = True, include_shotnumber = False, shot_indices = list(range(55)))
#X_train_time = X_train[:, 0]
#X_train = X_train[:, 1:]
X_test, y_test, _ = get_training_data(include_time = True, include_shotnumber = False, shot_indices = list(range(56, 60)))
#X_test_time = X_test[:, 0]
#X_test = X_test[:, 1:]
X_train.shape

(571675, 20)

In [78]:
X_tensor_train = torch.tensor(X_train, dtype=torch.float32)
y_tensor_train = torch.tensor((y_train - 1).astype(int), dtype=torch.long)

X_tensor_test = torch.tensor(X_test, dtype=torch.float32)
y_tensor_test = torch.tensor((y_test - 1).astype(int), dtype=torch.long)

In [80]:
# Convert NumPy arrays to PyTorch tensors
features_tensor = torch.tensor(X_tensor_train, dtype=torch.float32)
labels_tensor = torch.tensor(y_tensor_train, dtype=torch.long)  # Assuming classification labels are integers

# Create a DataLoader for batching
dataset = TensorDataset(features_tensor, labels_tensor)
batch_size = 32  # You can adjust this according to your dataset size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


/var/folders/vq/vpp5gf_96v7f5w4frq2kpfn00000gn/T/ipykernel_1441/3614208288.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features_tensor = torch.tensor(X_tensor_train, dtype=torch.float32)
/var/folders/vq/vpp5gf_96v7f5w4frq2kpfn00000gn/T/ipykernel_1441/3614208288.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(y_tensor_train, dtype=torch.long)  # Assuming classification labels are integers


In [81]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
    
        # Unsqueeze to make hx and cx 2-D for unbatched input
        h0 = h0.squeeze(1)
        c0 = c0.squeeze(1)
    
        # Forward pass through LSTM layer
        out, _ = self.lstm(x.unsqueeze(1), (h0, c0))
    
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out



In [82]:
input_size = 20  # Number of features per time step
hidden_size = 64  # Number of LSTM units
num_layers = 1  # Number of LSTM layers
num_classes = 3  # Replace with the actual number of unique labels in your dataset
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [83]:
num_epochs = 100  # Adjust as needed

for epoch in range(num_epochs):
    show = True
    for inputs, labels in dataloader:
        print(inputs[:, 0])
        # Forward pass
        #outputs = model(inputs)
        #loss = criterion(outputs, labels)

        # Backpropagation and optimization
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
        if(show):
            #print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}')
            print(f'Epoch [{epoch + 1}/{num_epochs}], ')
            show = False
        break
        

print('Training complete!')


tensor([0.6500, 0.6501, 0.6502, 0.6503, 0.6504, 0.6505, 0.6506, 0.6507, 0.6508,
        0.6509, 0.6510, 0.6511, 0.6512, 0.6513, 0.6514, 0.6515, 0.6516, 0.6517,
        0.6518, 0.6519, 0.6520, 0.6521, 0.6522, 0.6523, 0.6524, 0.6525, 0.6526,
        0.6527, 0.6528, 0.6529, 0.6530, 0.6531])
Epoch [1/100], 
tensor([0.6500, 0.6501, 0.6502, 0.6503, 0.6504, 0.6505, 0.6506, 0.6507, 0.6508,
        0.6509, 0.6510, 0.6511, 0.6512, 0.6513, 0.6514, 0.6515, 0.6516, 0.6517,
        0.6518, 0.6519, 0.6520, 0.6521, 0.6522, 0.6523, 0.6524, 0.6525, 0.6526,
        0.6527, 0.6528, 0.6529, 0.6530, 0.6531])
Epoch [2/100], 
tensor([0.6500, 0.6501, 0.6502, 0.6503, 0.6504, 0.6505, 0.6506, 0.6507, 0.6508,
        0.6509, 0.6510, 0.6511, 0.6512, 0.6513, 0.6514, 0.6515, 0.6516, 0.6517,
        0.6518, 0.6519, 0.6520, 0.6521, 0.6522, 0.6523, 0.6524, 0.6525, 0.6526,
        0.6527, 0.6528, 0.6529, 0.6530, 0.6531])
Epoch [3/100], 
tensor([0.6500, 0.6501, 0.6502, 0.6503, 0.6504, 0.6505, 0.6506, 0.6507, 0.6508,
     

In [44]:
test_dataset = TensorDataset(X_tensor_test, y_tensor_test)
batch_size = 32  # You can adjust this according to your dataset size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [56]:

def multi_confusion_matrix(y_predicted, y_expected):
    """
    Function constructing a multiclass confusion matrix
    
    :param y_predicted: pd.Series containing the predicted labels
    :param y_expected: pd.Series containing the expected labels
    :return: pd.Dataframe containing the multiclass confusion matrix 
    with columns as true labels and rows are predicted labels
    """
    predicted_labels = y_predicted.unique()
    expected_labels = y_expected.unique()
    
    mtrx = pd.DataFrame(0, columns=expected_labels, index=expected_labels)
    
    for expected_label in expected_labels:
        for predicted_label in predicted_labels:
            count = sum((y_expected == expected_label) & (y_predicted == predicted_label))
            mtrx.at[predicted_label, expected_label] = count
    return mtrx

def diag(matrix):
    """
    Function retrieving the true values of a confusion matrix
    
    :param matrix: confusion matrix
    :return: list of all the diagonal values
    """
    if len(matrix.index) <= len(matrix.columns):
        zipped = zip(matrix.index, matrix.columns[:len(matrix.index)])
    else:
        zipped = zip(matrix.index[:len(matrix.columns)], matrix.columns)
    
    diag = []
    for idx,col in zipped:
        diag.append(matrix.at[idx,col])
    return diag

def multi_accuracy(matrix):
    """
    Function computing the accuracy associated with a confusion matrix
    :param matrix: confusion matrix
    :return: precision score
    """
    t = sum(diag(matrix))
    all = matrix.sum().sum()
    return t / all
    
def multi_precision(matrix):
    """
    Function computing the precision associated with a confusion matrix
    :param matrix: confusion matrix
    :return: precision score
    """
    tp = diag(matrix)
    pred_p = matrix.sum(axis=1)
    return (tp / pred_p).fillna(0)
def multi_recall(matrix):
    """
    Function computing the recall associated with a confusion matrix
    :param matrix: confusion matrix
    :return: recall score
    """
    tp = diag(matrix)
    acutal_p = matrix.sum(axis=0)
    return (tp / acutal_p).fillna(0)

def multi_f1_score(matrix):
    """
    Function computing the F1-score associated with a confusion matrix
    :param matrix: confusion matrix
    :return: F1-score
    """
    recall = multi_recall(matrix)
    precision = multi_precision(matrix)
    
    multiplied = recall.multiply(precision)
    summed =recall + precision
    return (2 * multiplied / summed).fillna(0)

In [57]:
import pandas as pd

model.eval()
all_predicted = []
all_labels = []

with torch.no_grad():
    for shots, labels in test_loader:
        outputs = model(shots)
        _, predicted = torch.max(outputs.data, 1)
        all_predicted.extend(predicted.numpy())
        all_labels.extend(labels.numpy())

# Convert to pandas Series for compatibility with your functions
y_predicted = pd.Series(all_predicted)
y_true = pd.Series(all_labels)

# Assuming your multi_confusion_matrix, multi_f1_score, and multi_accuracy functions accept pandas Series
mtrx_test = multi_confusion_matrix(y_predicted + 1, y_true + 1)  # Adjust indices if necessary
f1_score = multi_f1_score(mtrx_test)
accuracy = multi_accuracy(mtrx_test)

# Print the evaluation metrics
print(f'F1 Score: {f1_score}')
print(f'Accuracy: {accuracy}')


F1 Score: 3    0.845849
1    0.866579
2    0.763901
dtype: float64
Accuracy: 0.8321216430062152
